In [1]:
# -*- coding: utf-8 -*- 
import os 
import argparse
import re 
import time 

from modules import folderMoniter, patternRecognizer, metaExtracter
from modules import urlDownload, dropboxInteractor, note_modified

In [2]:
# init 
pattern_recog = patternRecognizer(r'- \[.*\]')  # 检测 - [DOI], 或者- [arxivId]
meta_extracter = metaExtracter()

In [5]:
md_file = r"C:\Users\dell\Desktop\Notes&Papers\test.md"

In [10]:
with open(md_file, 'r', encoding='utf-8') as f:
    content = f.read()
m = pattern_recog.findall(content)
print(m)

if m:
    replace_dict = dict()
    for literature in m:
        literature_id = literature.split('[')[-1].split(']')[0]
        try:
            bib_dict = meta_extracter.id2bib(literature_id)
            print(bib_dict)
            
#             if "pdf_link" in bib_dict.keys():
#                 pdf_dict = url_download.fetch(bib_dict["pdf_link"])
#                 if not pdf_dict:
#                     pdf_dict = url_download.fetch(literature_id)
#             else:
#                 pdf_dict = url_download.fetch(literature_id)
#             print(pdf_dict)
            
            # Upload attachment and generate shared link
            if "\n" in bib_dict["title"]:
                bib_dict["title"] = re.sub(r' *\n *', ' ', bib_dict["title"])
                
            pdf_shared_link = ""
                
            if 'cited_count' in bib_dict.keys():
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([link]({})), (Citations **{}**).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url'], bib_dict["cited_count"]
                    )
            else:
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([link]({})), (Citations ).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url']
                    )
                
            print(replaced_literature)
            replace_dict[literature] = replaced_literature
        except:
            print("有错误")
            # replace_dict[literature] = literature
            
    # Modified note
    
    note_modified(pattern_recog, md_file, **replace_dict)


['- [2111.07999]']
{'journal': 'arxiv', 'url': 'http://arxiv.org/abs/2111.07999v1', 'title': 'Adversarial Skill Chaining for Long-Horizon Robot Manipulation via\n  Terminal State Regularization', 'year': '2021', 'author': 'Youngwoon Lee and Joseph J. Lim and Anima Anandkumar and Yuke Zhu', 'ENTRYTYPE': 'article'}
- **Adversarial Skill Chaining for Long-Horizon Robot Manipulation via Terminal State Regularization**. Youngwoon Lee et.al. **arxiv**, **2021**, ([pdf]())([link](http://arxiv.org/abs/2111.07999v1)), (Citations ).
